<a href="https://colab.research.google.com/github/kurilchenkoyu/pet_adoption/blob/main/Film_ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Предсказание рейтингов фильмов**





## Импортирование необходимых библиотек

In [ ]:
import pandas as pd
import numpy as np

## Импортирование файлов с Kaggle

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"anfgln","key":"04c661c3f1e8d8a5173a5d8c1cbb6cff"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c 'films-rating-prediction'

films-rating-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
! unzip films-rating-prediction.zip -d data

Archive:  films-rating-prediction.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_samples.csv   
  inflating: data/train_samples.csv  


In [ ]:
train_df = pd.read_csv("data/train_samples.csv", lineterminator='\n', index_col=None, header=0, sep=',')
test_df = pd.read_csv("data/train_samples.csv", lineterminator='\n', index_col=None, header=0, sep=',')

## Первичный анализ данных и предобработка

### Просмотр данных

In [ ]:
train_df.head()

,id,Title,Rated,Released,Runtime,Director,Writer,Actors,Plot,Poster,...,Budget,Genres,Original_language,Overview,Poster_path,Production_companies,Production_countries,Status,Tagline,Target
0,film13,Forrest Gump,PG-13,06 Jul 1994,142 min,Robert Zemeckis,"Winston Groom, Eric Roth","Tom Hanks, Robin Wright, Gary Sinise",The history of the United States from the 1950...,https://m.media-amazon.com/images/M/MV5BNWIwOD...,...,55000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,A man with a low IQ has accomplished great thi...,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,"[{'id': 4, 'logo_path': '/gz66EfNoYPqHTYI4q9UE...","[{'iso_3166_1': 'US', 'name': 'United States o...",Released,The world will never be the same once you've s...,1011.774083
1,film18,The Fifth Element,PG-13,09 May 1997,126 min,Luc Besson,"Luc Besson, Robert Mark Kamen","Bruce Willis, Milla Jovovich, Gary Oldman","In the colorful future, a cab driver unwitting...",https://m.media-amazon.com/images/M/MV5BZWFjYm...,...,90000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,"In 2257, a taxi driver is unintentionally give...",/fPtlCO1yQtnoLHOwKtWz7db6RGU.jpg,"[{'id': 9, 'logo_path': '/nda3dTUYdDrJ6rZqBpYv...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",Released,There is no future without it.,866.485451
2,film88,Dirty Dancing,PG-13,21 Aug 1987,100 min,Emile Ardolino,Eleanor Bergstein,"Patrick Swayze, Jennifer Grey, Jerry Orbach",Spending the summer at a Catskills resort with...,https://m.media-amazon.com/images/M/MV5BYjM0Mz...,...,6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",en,Expecting the usual tedium that accompanies a ...,/9Jw6jys7q9gjzVX5zm1z0gC8gY9.jpg,"[{'id': 137, 'logo_path': None, 'name': 'Great...","[{'iso_3166_1': 'US', 'name': 'United States o...",Released,First dance. First love. The time of your life.,727.882495
3,film95,Armageddon,PG-13,01 Jul 1998,151 min,Michael Bay,"Jonathan Hensleigh, J.J. Abrams, Tony Gilroy","Bruce Willis, Billy Bob Thornton, Ben Affleck",After discovering that an asteroid the size of...,https://m.media-amazon.com/images/M/MV5BMGM0Nz...,...,140000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,When an asteroid threatens to collide with Ear...,/eTM3qtGhDU8cvjpoa6KEt5E2auU.jpg,"[{'id': 9195, 'logo_path': '/ou5BUbtulr6tIt699...","[{'iso_3166_1': 'US', 'name': 'United States o...",Released,For love. For honor. For mankind.,795.618813
4,film123,The Lord of the Rings,PG,15 Nov 1978,132 min,Ralph Bakshi,"Chris Conkling, Peter S. Beagle, J.R.R. Tolkien","Christopher Guard, William Squire, Michael Sch...",The Fellowship of the Ring embark on a journey...,https://m.media-amazon.com/images/M/MV5BOGExMj...,...,4000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",en,The Fellowship of the Ring embark on a journey...,/liW0mjvTyLs7UCumaHhx3PpU4VT.jpg,"[{'id': 286, 'logo_path': None, 'name': 'Fanta...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",Released,Fantasy...beyond your imagination,714.239726


In [ ]:
train_df.columns

Index(['id', 'Title', 'Rated', 'Released', 'Runtime', 'Director', 'Writer',
       'Actors', 'Plot', 'Poster', 'DVD', 'Budget', 'Genres',
       'Original_language', 'Overview', 'Poster_path', 'Production_companies',
       'Production_countries', 'Status', 'Tagline', 'Target'],
      dtype='object')

In [ ]:
train_df.duplicated().sum()

0

In [ ]:
test_df.duplicated().sum()

0

### Заполнение пропусков

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19257 entries, 0 to 19256
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    19257 non-null  object 
 1   Title                 19257 non-null  object 
 2   Rated                 7532 non-null   object 
 3   Released              17602 non-null  object 
 4   Runtime               18718 non-null  object 
 5   Director              18998 non-null  object 
 6   Writer                17394 non-null  object 
 7   Actors                18512 non-null  object 
 8   Plot                  17743 non-null  object 
 9   Poster                18513 non-null  object 
 10  DVD                   5925 non-null   object 
 11  Budget                19257 non-null  int64  
 12  Genres                19257 non-null  object 
 13  Original_language     19257 non-null  object 
 14  Overview              18252 non-null  object 
 15  Poster_path        

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19257 entries, 0 to 19256
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    19257 non-null  object 
 1   Title                 19257 non-null  object 
 2   Rated                 7532 non-null   object 
 3   Released              17602 non-null  object 
 4   Runtime               18718 non-null  object 
 5   Director              18998 non-null  object 
 6   Writer                17394 non-null  object 
 7   Actors                18512 non-null  object 
 8   Plot                  17743 non-null  object 
 9   Poster                18513 non-null  object 
 10  DVD                   5925 non-null   object 
 11  Budget                19257 non-null  int64  
 12  Genres                19257 non-null  object 
 13  Original_language     19257 non-null  object 
 14  Overview              18252 non-null  object 
 15  Poster_path        

In [ ]:
# Подсчет числа пропущенных значений в каждой колонке тренировочной выборки
missing_values = train_df.isnull().sum()

# Подсчет общего числа значений в каждой колонке
total_values = train_df.shape[0]

# Вычисление процента пропущенных значений в каждой колонке
percent_missing = (missing_values / total_values) * 100

missing_data = pd.DataFrame({'Total Missing': missing_values, 'Percent Missing': percent_missing})

print(missing_data)

                      Total Missing  Percent Missing
id                                0         0.000000
Title                             0         0.000000
Rated                         11725        60.886950
Released                       1655         8.594277
Runtime                         539         2.798982
Director                        259         1.344965
Writer                         1863         9.674404
Actors                          745         3.868723
Plot                           1514         7.862076
Poster                          744         3.863530
DVD                           13332        69.231968
Budget                            0         0.000000
Genres                            0         0.000000
Original_language                 0         0.000000
Overview                       1005         5.218881
Poster_path                    1485         7.711482
Production_companies              0         0.000000
Production_countries              0         0.

In [ ]:
# Подсчет числа пропущенных значений в каждой колонке тестовой выборки
missing_values = test_df.isnull().sum()

# Подсчет общего числа значений в каждой колонке
total_values = test_df.shape[0]

# Вычисление процента пропущенных значений в каждой колонке
percent_missing = (missing_values / total_values) * 100

missing_data = pd.DataFrame({'Total Missing': missing_values, 'Percent Missing': percent_missing})

print(missing_data)

                      Total Missing  Percent Missing
id                                0         0.000000
Title                             0         0.000000
Rated                         11725        60.886950
Released                       1655         8.594277
Runtime                         539         2.798982
Director                        259         1.344965
Writer                         1863         9.674404
Actors                          745         3.868723
Plot                           1514         7.862076
Poster                          744         3.863530
DVD                           13332        69.231968
Budget                            0         0.000000
Genres                            0         0.000000
Original_language                 0         0.000000
Overview                       1005         5.218881
Poster_path                    1485         7.711482
Production_companies              0         0.000000
Production_countries              0         0.

In [ ]:
# Удаление признаков DVD, Poster, Poster_path: считаем, что они не влияют на результат
# Tagline - слишком много пропусков
train_df.drop(columns=["DVD", "Poster", "Poster_path", "Tagline"], inplace=True)
test_df.drop(columns=["DVD", "Poster", "Poster_path", "Tagline"], inplace=True)

In [ ]:
# Заполнение пропущенных значений в колонке 'Released' предыдущим значением
train_df['Released'].fillna(method='ffill', inplace=True)
test_df['Released'].fillna(method='ffill', inplace=True)

In [ ]:
train_df["Rated"].unique()

array(['PG-13', 'PG', nan, 'GP', 'Approved', 'Passed', 'TV-PG', 'TV-14',
       'G', 'M/PG', 'TV-G', 'M', 'TV-Y7-FV', 'TV-Y7', 'TV-Y', 'APPROVED',
       'E', '14', 'II', 'PD', 'U', '15', '16+', 'F', 'S', 'Open', '16'],
      dtype=object)

In [ ]:
# Словарь, сопоставляющий существующие рейтинги с рейтингами MPAA
rating_mapping = {
    'PG-13': 'PG-13',
    'PG': 'PG',
    'GP': 'PG',
    'Approved': 'PG',
    'Passed': 'PG',
    'TV-PG': 'PG',
    'TV-14': 'PG-13',
    'G': 'G',
    'M/PG': 'PG',
    'TV-G': 'G',
    'M': 'R',
    'TV-Y7-FV': 'PG',
    'TV-Y7': 'PG',
    'TV-Y': 'G',
    'APPROVED': 'PG',
    'E': 'G',
    '14': 'PG-13',
    'II': 'PG',
    'PD': 'PG',
    'U': 'G',
    '15': 'PG-13',
    '16+': 'R',
    'F': 'PG',
    'S': 'PG',
    'Open': 'PG',
    '16': 'R'
}

# Применить сопоставление к столбцу
train_df['Rated'] = train_df['Rated'].map(rating_mapping)
test_df['Rated'] = test_df['Rated'].map(rating_mapping)

# Заполнение пропусков
most_common_rating = train_df['Rated'].mode()[0]
train_df['Rated'].fillna(most_common_rating, inplace=True)
test_df['Rated'].fillna(most_common_rating, inplace=True)

In [ ]:
train_df['Director'].fillna('Unknown', inplace=True)
test_df['Director'].fillna('Unknown', inplace=True)

train_df['Writer'].fillna('Unknown', inplace=True)
test_df['Writer'].fillna('Unknown', inplace=True)

train_df['Actors'].fillna('Unknown', inplace=True)
test_df['Actors'].fillna('Unknown', inplace=True)

train_df['Plot'].fillna('Unknown', inplace=True)
test_df['Plot'].fillna('Unknown', inplace=True)

train_df['Overview'].fillna('Unknown', inplace=True)
test_df['Overview'].fillna('Unknown', inplace=True)

train_df['Overview'].fillna('Unknown', inplace=True)
test_df['Overview'].fillna('Unknown', inplace=True)


In [ ]:
# Преобразование колонки Runtime
train_df['Runtime'] = train_df['Runtime'].str[:-4]
test_df['Runtime'] = test_df['Runtime'].str[:-4]

In [ ]:
train_df['Runtime'] = pd.to_numeric(train_df['Runtime'], errors='coerce')
test_df['Runtime'] = pd.to_numeric(test_df['Runtime'], errors='coerce')

In [ ]:
# Заполняем пропуски средним значением
mean_runtime = train_df['Runtime'].mean()

train_df['Runtime'].fillna(mean_runtime, inplace=True)
test_df['Runtime'].fillna(mean_runtime, inplace=True)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19257 entries, 0 to 19256
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    19257 non-null  object 
 1   Title                 19257 non-null  object 
 2   Rated                 19257 non-null  object 
 3   Released              19257 non-null  object 
 4   Runtime               19257 non-null  float64
 5   Director              19257 non-null  object 
 6   Writer                19257 non-null  object 
 7   Actors                19257 non-null  object 
 8   Plot                  19257 non-null  object 
 9   Budget                19257 non-null  int64  
 10  Genres                19257 non-null  object 
 11  Original_language     19257 non-null  object 
 12  Overview              19257 non-null  object 
 13  Production_companies  19257 non-null  object 
 14  Production_countries  19257 non-null  object 
 15  Status             

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19257 entries, 0 to 19256
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    19257 non-null  object 
 1   Title                 19257 non-null  object 
 2   Rated                 19257 non-null  object 
 3   Released              19257 non-null  object 
 4   Runtime               19257 non-null  float64
 5   Director              19257 non-null  object 
 6   Writer                19257 non-null  object 
 7   Actors                19257 non-null  object 
 8   Plot                  19257 non-null  object 
 9   Budget                19257 non-null  int64  
 10  Genres                19257 non-null  object 
 11  Original_language     19257 non-null  object 
 12  Overview              19257 non-null  object 
 13  Production_companies  19257 non-null  object 
 14  Production_countries  19257 non-null  object 
 15  Status             

Все пропуски заполнены!

### Работа с признаками сложных структур, feature engineering

In [ ]:
def split_values(row):
    split_row = row.split(', ')
    split_row.extend(['No'] * (3 - len(split_row)))
    new_series = pd.Series(split_row)
    print(new_series)
    return pd.Series(split_row)

train_df[['Director1', 'Director2', 'Director3']] = train_df['Director'].apply(split_values)
train_df[['Writer1', 'Writer2', 'Writer3']] = train_df['Writer'].apply(split_values)
train_df[['Actor1', 'Actor2', 'Actor3']] = train_df['Actors'].apply(split_values)

Streaming output truncated to the last 5000 lines.
0    J.D. Brown
1            No
2            No
dtype: object
0    Gianni Puccini
1                No
2                No
dtype: object
0    Efren Jarlego
1               No
2               No
dtype: object
0    Aloy Adlawan
1              No
2              No
dtype: object
0     Greg Tiernan
1    John Gilluley
2               No
dtype: object
0       Derek Jarman
1    Michael Kostiff
2       John Maybury
dtype: object
0    Thomas Kronthaler
1                   No
2                   No
dtype: object
0    Rainer Erler
1              No
2              No
dtype: object
0    Dominic Murphy
1                No
2                No
dtype: object
0       David Baas
1    John Gilluley
2               No
dtype: object
0    Tim Burton
1            No
2            No
dtype: object
0    Nick Nostro
1             No
2             No
dtype: object
0    Mats Helge Olsson
1                   No
2                   No
dtype: object
0    Julien Temple
1

ValueError: Columns must be same length as key

## Визуализация данных

## Кодирование признаков

### Кодирование категориальных признаков

### Токенизация и векторизация

## Масштабирование признаков

## Модели и результаты

### Важность признаков

## Проверка и оценка качества

## Изменения

## Итоговая модель


Модель